<div align="center">

<a id="inicio"></a>
 
# 📊 Aplicaciones de PCA en Ingeniería Electrónica

## 🔬 Análisis de Componentes Principales para Problemas Electrónicos

**Casos prácticos de Machine Learning aplicado a Electrónica**

</div>

---

## 📋 Contenido del Notebook

Este notebook presenta **un ejemplo introductorio con el dataset Iris** seguido de **tres aplicaciones prácticas** del Análisis de Componentes Principales (PCA) en el dominio de la ingeniería electrónica:

[[ libs ]](#libs)

### 🎯 Aplicaciones

- **[🔍 PCA en el Dataset Iris](#iris)**
   - Visualizar las muestras antes y después de la transformación PCA
   - Análisis de varianza explicada
   - Gráfico de codo para selección de componentes

1. **[🔊 Limpieza de Ruido en Señales](#limpieza)**
   - Filtrado de señales electrónicas contaminadas
   - Mejora de SNR (Signal-to-Noise Ratio)
   - Análisis espectral comparativo

2. **[⚡ Detección de Fallas en Circuitos](#fallas)**
   - Identificación de componentes defectuosos
   - Análisis de anomalías en características eléctricas
   - Clasificación con Machine Learning

3. **[📸 Compresión de Imágenes de Componentes](#compresion)**
   - Reducción dimensional de imágenes
   - Optimización de almacenamiento
   - Reconstrucción con pérdida controlada

- **[📌 Buenas Prácticas al Usar PCA en Electrónica](#recomendaciones)**

- **[📚 Recursos Adicionales](#referencias)**

---

### 🛠️ Herramientas Utilizadas

- **NumPy**: Cálculo numérico y procesamiento de señales
- **Pandas**: Manipulación de datos
- **Matplotlib**: Visualización de resultados
- **Scikit-learn**: Implementación de PCA y algoritmos ML
  - `PCA`: Análisis de Componentes Principales
  - `StandardScaler`: Normalización de datos
  - `IsolationForest`: Detección de anomalías

---

### 📚 Conceptos Clave

- Descomposición en valores singulares (SVD)
- Reducción de dimensionalidad
- Varianza explicada
- Transformación lineal
- Detección de anomalías
- Procesamiento de señales digitales


---

<a id="libs"></a>
## 📦 Importación de Librerías y Configuración [↑](#inicio)

---

Configuramos el entorno de trabajo importando las librerías necesarias y suprimiendo warnings para mejor legibilidad.


In [ ]:
# Cargar las librerías necesarias
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

# ============================================================================
# SUPRIMIR WARNINGS
# ============================================================================
# Suprimir advertencias para visualización limpia
warnings.filterwarnings('ignore')

# Configuración de NumPy para errores numéricos
np.seterr(divide='ignore', invalid='ignore', over='ignore')

# ============================================================================
# CONFIGURACIÓN DE MATPLOTLIB
# ============================================================================
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.style.use('seaborn-v0_8')

print("✅ Librerías importadas correctamente")
print("✅ Warnings suprimidos")

---

<a id="iris"></a>
# 🔍 Ejemplo Introductorio: PCA en el Dataset Iris [↑](#inicio)

---

## 🎯 Introducción

Antes de aplicar PCA a problemas de ingeniería electrónica, comenzamos con un ejemplo clásico: el **dataset Iris**. Este conjunto de datos contiene mediciones de 150 flores de iris de tres especies diferentes.

## 📊 Objetivos

- Visualizar los datos originales en 4 dimensiones
- Aplicar PCA para reducir a 2 componentes principales
- Analizar la varianza explicada
- Comparar visualizaciones antes y después de PCA


In [ ]:
# Cargar el dataset Iris
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

# Crear un DataFrame
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

# Seleccionar 10 muestras por clase (solo para mejor visualizacion de las muestras)
df_parcial = df.groupby('target').apply(lambda x: x.sample(n=10, random_state=42)).reset_index(drop=True)
df_parcial['class_name'] = df_parcial['target'].map(dict(enumerate(target_names)))

# Obtener la paleta de colores usada por Seaborn
palette = sns.color_palette("Set1", n_colors=3)

# Graficar dispersión de los dos primeros atributos originales
plt.figure(figsize=(8, 6))
for i, label in enumerate(target_names):
    plt.scatter(df_parcial.loc[df_parcial['target'] == i, feature_names[0]],
                df_parcial.loc[df_parcial['target'] == i, feature_names[1]],
                label=label,
                color=palette[i])  # usar el mismo color que sns
plt.xlabel(feature_names[0])
plt.ylabel(feature_names[1])
plt.title("Diagrama de dispersión de los dos primeros atributos (originales)")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="class_name")
plt.grid(True)
plt.tight_layout()
plt.savefig("nuevo_iris_antes_del_PCA.png", format="png", dpi=100)
plt.show()

# Graficar todos los 6 diagramas de dispersión 2 a 2
sns.set(style="ticks")
sns.pairplot(df_parcial, hue="class_name", vars=feature_names, plot_kws={'alpha': 0.8, 's': 60}, palette='Set1')
plt.suptitle("Todos los diagramas de dispersión (atributos originales)", y=1.02)
plt.savefig("nuevo_iris_todos_antes_del_PCA.png", format="png", dpi=100)
plt.show()

# Crear el modelo PCA para poder obtener todos los componentes posibles (que este exemplo es 4)
pca = PCA()
# fit realiza la descomposición SVD de la matriz centrada
# transform proyecta los datos sobre los componentes principales (producto matricial XV)
X_pca = pca.fit_transform(df_parcial[feature_names])
print('Componentes principales. sklearn entrega la tranpuesta de V:\n', pca.components_)


# Crear DataFrame con solo los 2 primeros componentes, para graficar las muestras en el nuevo 2D
df_pca = pd.DataFrame(X_pca[:, :2], columns=['PCA1', 'PCA2'])
df_pca['target'] = df_parcial['target']

plt.figure(figsize=(8,6))
for i, label in enumerate(target_names):
    plt.scatter(df_pca.loc[df_pca['target'] == i, 'PCA1'],
                df_pca.loc[df_pca['target'] == i, 'PCA2'],
                label=label,
                color=palette[i])
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.title("Diagrama de dispersión usando PCA (2 componentes principales)")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="class_name")
plt.grid(True)
plt.tight_layout()
plt.savefig("nuevo_iris_despues_del_PCA.png", format="png", dpi=100)
plt.show()

#Proporción de la varianza explicada por cada componente
explained_var = pca.explained_variance_ratio_

print("Varianza explicada por cada componente:                       ", np.round(pca.explained_variance_ratio_, 4))
print("Varianza acumulada con los 2 primeros componentes principales:", round(np.sum(pca.explained_variance_ratio_[:2]), 4))
print("Varianza total acumulada:                                     ", round(np.sum(pca.explained_variance_ratio_), 4))

# Índices de los componentes (1, 2, 3, ...)
components = np.arange(1, len(explained_var) + 1)
plt.figure(figsize=(8, 5))
# Barras: varianza explicada por componente
plt.bar(components, explained_var, alpha=0.5, align='center', label='Varianza por componente')
# Linea: varianza acumulada
plt.plot(components, np.cumsum(explained_var), marker='o', linestyle='--', color='b', label='Varianza acumulada')
plt.xlabel('Número de Componentes Principais')
plt.ylabel('Proporción de la Varianza Explicada')
plt.title('Gráfico de codo - PCA')
plt.xticks(components)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Con todos los componentes principales')
plt.grid(True)
plt.tight_layout()
plt.savefig("nuevo_iris_grafico_codo_PCA.png", format="png", dpi=100)
plt.show()


---

<a id="limpieza"></a>
# 🔊 Aplicación 1: Limpieza de Ruido en Señales Electrónicas [↑](#inicio)

---

## 🎯 Problema

Las señales electrónicas están frecuentemente contaminadas con ruido que degrada la calidad de la información transmitida. El ruido puede provenir de:
- 🌡️ Ruido térmico (Johnson-Nyquist)
- ⚡ Interferencia electromagnética (EMI)
- 🔌 Ruido de cuantización en ADC/DAC
- 📡 Crosstalk entre señales

## 💡 Solución con PCA

Utilizamos PCA para separar las componentes principales de la señal (información útil) del ruido (componentes de baja varianza), aplicando:
1. Segmentación de la señal en ventanas temporales
2. Extracción de componentes principales
3. Reconstrucción usando solo componentes significativos


In [ ]:
# ============================================================================
# FUNCIÓN: Generar señal sintética con ruido
# ============================================================================
def generar_señal_electrónica(t, frecuencia=50, amplitud=1, ruido=0.3):
    """
    Genera una señal electrónica sintética con componentes armónicos y ruido.

    Parámetros:
        t: Array de tiempo (segundos)
        frecuencia: Frecuencia fundamental de la señal (Hz)
        amplitud: Amplitud máxima de la señal (V)
        ruido: Factor de ruido a agregar (valor entre 0 y 1)

    Retorna:
        señal_limpia: Señal sin ruido (solo frecuencia fundamental + armónicos)
        señal_ruidosa: Señal con ruido gaussiano y de alta frecuencia añadido
    """

    # Componente principal: señal senoidal a la frecuencia fundamental
    señal_limpia = amplitud * np.sin(2 * np.pi * frecuencia * t)

    # Armónicos: componentes a frecuencias múltiplos de la fundamental
    # Tercer armónico (3x frecuencia fundamental, 30% de amplitud)
    armónico_1 = 0.3 * amplitud * np.sin(2 * np.pi * frecuencia * t * 3)
    # Quinto armónico (5x frecuencia fundamental, 10% de amplitud)
    armónico_2 = 0.1 * amplitud * np.sin(2 * np.pi * frecuencia * t * 5)

    # Ruido gaussiano: distribución normal con media 0 y desviación estándar 1
    # Simula ruido térmico y de cuantización
    ruido_gaussiano = ruido * np.random.normal(0, 1, len(t))

    # Ruido de alta frecuencia: simula interferencia electromagnética
    # Frecuencia 50 veces mayor que la fundamental (2.5 kHz si f=50Hz)
    ruido_af = 0.2 * ruido * amplitud * np.sin(2 * np.pi * frecuencia * t * 50)

    # Señal final: combinación de componentes limpias + ruido
    señal_ruidosa = señal_limpia + armónico_1 + armónico_2 + ruido_gaussiano + ruido_af

    return señal_limpia, señal_ruidosa

# ============================================================================
# GENERACIÓN DE SEÑALES PARA ANÁLISIS
# ============================================================================

# Crear vector de tiempo: 1000 muestras distribuidas uniformemente en 1 segundo
# Esto da una tasa de muestreo de 1000 Hz
tiempo = np.linspace(0, 1, 1000)

# Generar las señales limpia y ruidosa
señal_limpia, señal_ruidosa = generar_señal_electrónica(tiempo)

# ============================================================================
# ANÁLISIS ESPECTRAL MEDIANTE TRANSFORMADA DE FOURIER
# ============================================================================

# Transformada de Fourier: convierte señal del dominio temporal al frecuencial
fft_limpia = np.fft.fft(señal_limpia)      # FFT de la señal limpia
fft_ruidosa = np.fft.fft(señal_ruidosa)    # FFT de la señal con ruido

# Calcular las frecuencias correspondientes a cada componente de la FFT
# El segundo argumento es el período de muestreo (diferencia entre muestras consecutivas)
frecuencias = np.fft.fftfreq(len(tiempo), tiempo[1] - tiempo[0])

In [ ]:
# Tamaño de las señales
print(f"    Señal limpia: {señal_limpia.shape}, Señal ruidosa: {señal_ruidosa.shape}")

In [ ]:
# ============================================================================
# VISUALIZACIÓN: Señales originales y análisis espectral
# ============================================================================
def gráficos():
    """Visualiza las señales en dominio temporal y frecuencial."""

    plt.figure(figsize=(15, 10))

    # ========================================================================
    # GRÁFICO 1: Señales en dominio temporal
    # ========================================================================
    plt.subplot(2, 2, 1)
    plt.plot(tiempo, señal_limpia, 'b-', linewidth=2, label='Señal limpia')
    plt.plot(tiempo, señal_ruidosa, 'r-', alpha=0.7, label='Señal ruidosa')
    plt.title('Señal Electrónica: Original vs Con Ruido')
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Amplitud (V)')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # ========================================================================
    # GRÁFICO 2: Análisis espectral (dominio frecuencial)
    # ========================================================================
    plt.subplot(2, 2, 2)
    # Mostrar solo frecuencias positivas (la mitad del espectro)
    plt.plot(frecuencias[:len(frecuencias)//2],
             np.abs(fft_limpia[:len(frecuencias)//2]),
             'b-', label='Espectro Original')
    plt.plot(frecuencias[:len(frecuencias)//2],
             np.abs(fft_ruidosa[:len(frecuencias)//2]),
             'r-', alpha=0.7, label='Espectro con Ruido')
    plt.title('Análisis Espectral')
    plt.xlabel('Frecuencia (Hz)')
    plt.ylabel('Magnitud')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # ========================================================================
    # CÁLCULO DE SNR (Relación Señal/Ruido)
    # ========================================================================
    # SNR en dB: 20*log10(potencia_señal / potencia_ruido)
    snr_db = 20*np.log10(np.std(señal_limpia)/np.std(señal_ruidosa - señal_limpia))

    print(f"SNR (Signal-to-Noise Ratio) estimado: {snr_db:.2f} dB")
    print(f"Relación entre la potencia de la señal y la potencia del ruido")

gráficos()

In [ ]:
# ============================================================================
# FUNCIÓN: Reducción de ruido mediante PCA
# ============================================================================
def reducir_ruido_con_pca(señal, n_ventanas=50, n_componentes=5):
    """
    Reduce el ruido de una señal mediante PCA aplicado a ventanas deslizantes.

    El método divide la señal en ventanas y aplica PCA para identificar y conservar
    solo los componentes principales (que contienen la señal útil), eliminando
    componentes de menor varianza (que típicamente corresponden al ruido).

    Parámetros:
        señal: Array con los valores de la señal ruidosa
        n_ventanas: Número de ventanas en que se divide la señal (default: 50)
        n_componentes: Número de componentes principales a conservar (default: 5)

    Retorna:
        señal_filtrada: Señal reconstruida con ruido reducido
        varianza_explicada: Array con la varianza explicada por cada componente
    """

    # ========================================================================
    # PASO 1: Dividir la señal en ventanas
    # ========================================================================
    # Calcular el tamaño de cada ventana (ej: 1000 muestras / 50 ventanas = 20 muestras/ventana)
    tamaño_ventana = len(señal) // n_ventanas

    # Lista para almacenar cada segmento de la señal
    ventanas = []

    # Dividir la señal en segmentos (ventanas) no solapados
    for i in range(n_ventanas):
        índice_inicio = i * tamaño_ventana
        índice_fin = índice_inicio + tamaño_ventana

        # Verificar que no excedamos el tamaño de la señal
        if índice_fin <= len(señal):
            ventanas.append(señal[índice_inicio:índice_fin])

    # ========================================================================
    # PASO 2: Convertir a matriz para aplicar PCA
    # ========================================================================
    # Cada fila representa una ventana, cada columna una muestra temporal
    # Matriz resultante: (n_ventanas x tamaño_ventana)
    matriz_ventanas = np.array(ventanas)

    # Normalizar cada ventana (restar media de cada ventana)
    # Esto evita problemas numéricos en el PCA
    matriz_ventanas = matriz_ventanas - np.mean(matriz_ventanas, axis=1, keepdims=True)

    # ========================================================================
    # PASO 3: Aplicar PCA para extraer componentes principales
    # ========================================================================
    # Crear modelo PCA que conservará solo los n componentes más importantes
    modelo_pca = PCA(n_components=n_componentes)

    # Ajustar el modelo: calcular componentes principales a partir de las ventanas
    modelo_pca.fit(matriz_ventanas)

    # ========================================================================
    # PASO 4: Transformar señal al espacio de componentes principales
    # ========================================================================
    # Proyectar las ventanas al nuevo espacio de menor dimensión
    # Matriz resultante: (n_ventanas x n_componentes)
    señal_transformada = modelo_pca.transform(matriz_ventanas)

    # ========================================================================
    # PASO 5: Reconstruir señal desde componentes principales
    # ========================================================================
    # Transformación inversa: de espacio PCA de vuelta al espacio original
    # Solo usa los n_componentes principales, eliminando componentes de ruido
    señal_reconstruida = modelo_pca.inverse_transform(señal_transformada)

    # ========================================================================
    # PASO 6: Ensamblar la señal completa desde las ventanas reconstruidas
    # ========================================================================
    # Crear array vacío del mismo tamaño que la señal original
    señal_filtrada = np.zeros_like(señal)

    # Colocar cada ventana reconstruida en su posición original
    for i, ventana in enumerate(señal_reconstruida):
        índice_inicio = i * tamaño_ventana
        índice_fin = índice_inicio + tamaño_ventana

        if índice_fin <= len(señal):
            señal_filtrada[índice_inicio:índice_fin] = ventana

    # Mostrar dimensiones en cada paso del procesamiento
    print(f"Señal: {señal.shape} -> {matriz_ventanas.shape} -> {señal_transformada.shape} ->"
          f"{señal_reconstruida.shape} -> {señal_filtrada.shape}")

    return señal_filtrada, modelo_pca.explained_variance_ratio_

# ============================================================================
# APLICAR FILTRADO PCA A LA SEÑAL RUIDOSA
# ============================================================================
señal_filtrada, varianza_explicada = reducir_ruido_con_pca(señal_ruidosa)

In [ ]:
# Para hacer pruebas


In [ ]:
# ============================================================================
# VISUALIZACIÓN: Resultados del filtrado PCA
# ============================================================================
def gráficos():
    """Visualiza el efecto del filtrado PCA en la reducción de ruido."""

    plt.figure(figsize=(15, 12))

    # ========================================================================
    # GRÁFICO 1: Comparación señal original vs señal ruidosa
    # ========================================================================
    plt.subplot(3, 2, 1)
    plt.plot(tiempo, señal_limpia,  'b-', linewidth=2, label='Señal Original')
    plt.plot(tiempo, señal_ruidosa, 'r:', alpha=0.5,   label='Señal con Ruido')
    plt.title('Comparación de Señales')
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Amplitud (V)')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # ========================================================================
    # GRÁFICO 2: Señal original vs señal filtrada con PCA
    # ========================================================================
    plt.subplot(3, 2, 2)
    plt.plot(tiempo, señal_limpia,   'b-', linewidth=2, label='Señal Original')
    plt.plot(tiempo, señal_filtrada, 'g:', linewidth=2, label='Señal Filtrada con PCA')
    plt.title('Resultado del Filtrado PCA')
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Amplitud (V)')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # ========================================================================
    # GRÁFICO 3: Análisis del ruido removido
    # ========================================================================
    plt.subplot(3, 2, 3)
    ruido_original = señal_ruidosa - señal_limpia
    ruido_residual = señal_filtrada - señal_limpia

    plt.plot(tiempo, ruido_original,  'r-', alpha=0.7, label='Ruido Original')
    plt.plot(tiempo, ruido_residual, 'g:', alpha=0.7, label='Ruido Residual')
    plt.title('Análisis del Ruido')
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Amplitud del Ruido (V)')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # ========================================================================
    # GRÁFICO 4: Varianza explicada por cada componente principal
    # ========================================================================
    plt.subplot(3, 2, 4)
    plt.bar(range(1, len(varianza_explicada)+1), varianza_explicada, color='blue')
    plt.title('Varianza Explicada por Componentes')
    plt.xlabel('Componente Principal')
    plt.ylabel('Varianza Explicada')
    plt.grid(True, alpha=0.3)

    # ========================================================================
    # GRÁFICO 5: Análisis espectral comparativo
    # ========================================================================
    # Calcular FFT de la señal filtrada
    fft_filtrada = np.fft.fft(señal_filtrada)

    plt.subplot(3, 2, 5)
    plt.plot(frecuencias[:len(frecuencias)//2],
             np.abs(fft_limpia[:len(frecuencias)//2]),
             'b-', label='Original')
    plt.plot(frecuencias[:len(frecuencias)//2],
             np.abs(fft_ruidosa[:len(frecuencias)//2]),
             'r-', alpha=0.5, label='Con Ruido')
    plt.plot(frecuencias[:len(frecuencias)//2],
             np.abs(fft_filtrada[:len(frecuencias)//2]),
             'g:', label='Filtrada')
    plt.title('Espectros Comparativos')
    plt.xlabel('Frecuencia (Hz)')
    plt.ylabel('Magnitud')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # ========================================================================
    # GRÁFICO 6: Métricas de calidad (SNR)
    # ========================================================================
    # Calcular SNR para señal ruidosa y filtrada
    snr_original = 20*np.log10(np.std(señal_limpia)/np.std(ruido_original))
    snr_filtrada = 20*np.log10(np.std(señal_limpia)/np.std(ruido_residual))
    mejora_snr = snr_filtrada - snr_original

    plt.subplot(3, 2, 6)
    metricas = ['SNR Original', 'SNR Filtrada', 'Mejora SNR']
    valores = [snr_original, snr_filtrada, mejora_snr]
    colores = ['blue', 'green', 'purple']
    plt.bar(metricas, valores, color=colores, alpha=0.7)
    plt.title('Métricas de Calidad')
    plt.ylabel('SNR (dB)')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # ========================================================================
    # RESUMEN NUMÉRICO
    # ========================================================================
    print(f"Métricas de Calidad del Filtrado:")
    print(f"=" * 50)
    print(f"SNR Original: {snr_original:.2f} dB")           # Señal ruidosa
    print(f"SNR Filtrada: {snr_filtrada:.2f} dB")           # Después de PCA
    print(f"Mejora en SNR: {mejora_snr:.2f} dB")            # Ganancia obtenida
    print(f"Varianza total explicada: {np.sum(varianza_explicada):.3f} ({np.sum(varianza_explicada)*100:.1f}%)")

gráficos()

---

<a id="fallas"></a>
# ⚡ Aplicación 2: Detección de Fallas en Circuitos Electrónicos [↑](#inicio)

---

## 🎯 Problema

Identificar componentes defectuosos en circuitos electrónicos mediante el análisis de sus características eléctricas medidas:
- 🔌 Resistencia, capacitancia, inductancia
- 📈 Impedancia y factor de calidad
- 🌡️ Temperatura y corriente de fuga
- 📊 Parámetros de operación fuera de especificación

## 💡 Solución con PCA

Combinamos PCA con algoritmos de detección de anomalías:
1. Reducción dimensional de características eléctricas
2. Identificación de patrones anómalos
3. Clasificación con Isolation Forest


In [ ]:
# ============================================================================
# FUNCIÓN: Generación de datos sintéticos de circuitos electrónicos
# ============================================================================
def generar_datos_circuito(n_muestras=200, probabilidad_falla=0.1):
    """
    Simula mediciones de características eléctricas de circuitos, incluyendo
    componentes normales y defectuosos.

    Genera datos sintéticos de 10 características eléctricas medidas en circuitos.
    Algunos circuitos tienen fallas introducidas aleatoriamente.

    Parámetros:
        n_muestras: Número total de circuitos a simular (default: 200)
        probabilidad_falla: Probabilidad de que un circuito tenga falla (default: 0.1 = 10%)

    Retorna:
        datos_normales: Array (n x 11) con circuitos normales + etiqueta [0]
        datos_fallas: Array (m x 11) con circuitos defectuosos + etiqueta [1]
    """

    # Listas para almacenar circuitos normales y defectuosos
    datos_normales = []
    datos_fallas = []

    # Generar n_muestras de circuitos
    for _ in range(n_muestras):
        # ====================================================================
        # CARACTERÍSTICAS ELÉCTRICAS TÍPICAS DE UN CIRCUITO
        # ====================================================================
        # Cada característica sigue una distribución normal con media y desviación

        resistencia = np.random.normal(100, 10)                 # 100Ω ± 10Ω (tolerancia 10%)
        capacitancia = np.random.normal(1e-6, 1e-7)             # 1µF ± 0.1µF
        inductancia = np.random.normal(1e-3, 1e-4)              # 1mH ± 0.1mH
        frecuencia_resonancia = np.random.normal(1000, 100)     # 1kHz ± 100Hz
        impedancia = np.random.normal(50, 5)                    # 50Ω ± 5Ω
        factor_calidad = np.random.normal(100, 10)              # Factor Q ± 10%
        temperatura = np.random.normal(25, 5)                   # 25°C ± 5°C
        humedad = np.random.normal(50, 10)                      # 50% ± 10%
        voltaje_ruido = np.random.normal(0, 0.1)                # 0V ± 0.1V (ruido)
        corriente_fuga = np.random.normal(1e-9, 1e-10)          # 1nA ± 0.1nA

        # Vector con todas las características del circuito
        caracteristicas = [
            resistencia, capacitancia, inductancia, frecuencia_resonancia,
            impedancia, factor_calidad, temperatura, humedad,
            voltaje_ruido, corriente_fuga
        ]

        # ====================================================================
        # INTRODUCIR FALLAS ALEATORIAS EN ALGUNOS CIRCUITOS
        # ====================================================================
        if np.random.random() < probabilidad_falla:
            # Seleccionar tipo de falla aleatoriamente
            tipo_falla = np.random.choice(['resistencia_alta', 'capacitancia_baja', 'ruido_excesivo'])

            # Modificar la característica correspondiente según el tipo de falla
            if tipo_falla == 'resistencia_alta':
                # Resistencia 2-5 veces mayor (componente quemado o degradado)
                caracteristicas[0] *= np.random.uniform(2, 5)

            elif tipo_falla == 'capacitancia_baja':
                # Capacitancia reducida al 10-50% (capacitor degradado o seco)
                caracteristicas[1] *= np.random.uniform(0.1, 0.5)

            elif tipo_falla == 'ruido_excesivo':
                # Ruido 5-20 veces mayor (mal filtrado o conexiones defectuosas)
                caracteristicas[8] *= np.random.uniform(5, 20)

            # Agregar etiqueta de falla [1] al final del vector
            datos_fallas.append(caracteristicas + [1])
        else:
            # Agregar etiqueta normal [0] al final del vector
            datos_normales.append(caracteristicas + [0])

    return np.array(datos_normales), np.array(datos_fallas)

# ============================================================================
# GENERAR DATASET DE CIRCUITOS ELECTRÓNICOS
# ============================================================================

# Generar datos sintéticos de circuitos normales y con fallas
datos_normales, datos_fallas = generar_datos_circuito()
# Resultado: dos matrices (n x 11) y (m x 11) donde n + m = 200

# ============================================================================
# PREPARAR DATOS PARA ANÁLISIS
# ============================================================================

# Combinar circuitos normales y defectuosos en un único dataset
todos_datos = np.vstack([datos_normales, datos_fallas])

# Separar características (X) y etiquetas (y)
X = todos_datos[:, :-1]  # Primeras 10 columnas: características eléctricas
y = todos_datos[:, -1]   # Última columna: etiqueta (0=normal, 1=falla)

# Definir nombres descriptivos para cada característica
nombres_caracteristicas = [
    'Resistencia (Ω)', 'Capacitancia (F)', 'Inductancia (H)', 'F Resonancia (Hz)',
    'Impedancia (Ω)', 'Factor Calidad', 'Temperatura (°C)', 'Humedad (%)',
    'Voltaje Ruido (V)', 'Corriente Fuga (A)'
]

# ============================================================================
# RESUMEN DEL DATASET GENERADO
# ============================================================================
print(f"Datos generados: {len(datos_normales)} normales, {len(datos_fallas)} con fallas")
print(f"Total de características: {X.shape[1]}")
print(f"Tasa de fallas: {len(datos_fallas)/len(todos_datos)*100:.1f}%")

In [ ]:
# Pruebas: más info de los datos
datos_normales.shape, datos_fallas.shape, todos_datos.shape, X.shape, y.shape

In [ ]:
# ============================================================================
# VISUALIZACIÓN: Análisis exploratorio de datos de circuitos
# ============================================================================
def gráficos():
    """Visualiza distribuciones y estadísticas de características de circuitos."""
    plt.figure(figsize=(15, 10))

    # Matriz de correlación
    plt.subplot(2, 3, 1)
    correlacion = np.corrcoef(X.T)
    im = plt.imshow(correlacion, cmap='coolwarm', aspect='auto')
    plt.colorbar(im)
    plt.title('Matriz de Correlación')
    plt.xlabel('Características')
    plt.ylabel('Características')

    # Distribución de características principales
    plt.subplot(2, 3, 2)
    plt.hist(X[y==0, 0], bins=20, alpha=0.7, label='Normal', color='blue')
    plt.hist(X[y==1, 0], bins=20, alpha=0.7, label='Falla', color='red')
    plt.xlabel('Resistencia (Ω)')
    plt.ylabel('Frecuencia')
    plt.title('Distribución de Resistencia')
    plt.legend()

    plt.subplot(2, 3, 3)
    plt.hist(X[y==0, 1], bins=20, alpha=0.7, label='Normal', color='blue')
    plt.hist(X[y==1, 1], bins=20, alpha=0.7, label='Falla', color='red')
    plt.xlabel('Capacitancia (F)')
    plt.ylabel('Frecuencia')
    plt.title('Distribución de Capacitancia')
    plt.legend()

    # Boxplot de características
    plt.subplot(2, 3, 4)
    datos_plot = [X[y==0, 8], X[y==1, 8]]  # Voltaje de ruido
    plt.boxplot(datos_plot, tick_labels=['Normal', 'Falla'])
    plt.ylabel('Voltaje Ruido (V)')
    plt.title('Distribución del Ruido')

    # Scatter plot de dos características
    plt.subplot(2, 3, 5)
    plt.scatter(X[y==0, 0], X[y==0, 1], alpha=0.6, label='Normal', color='blue')
    plt.scatter(X[y==1, 0], X[y==1, 1], alpha=0.6, label='Falla', color='red')
    plt.xlabel('Resistencia (Ω)')
    plt.ylabel('Capacitancia (F)')
    plt.title('Resistencia vs Capacitancia')
    plt.legend()

    # Estadísticas por tipo
    plt.subplot(2, 3, 6)
    estadisticas = ['Media', 'Desv. Est.', 'Min', 'Max']
    valores_normales = [np.mean(X[y==0, 0]), np.std(X[y==0, 0]), np.min(X[y==0, 0]), np.max(X[y==0, 0])]
    valores_fallas = [np.mean(X[y==1, 0]), np.std(X[y==1, 0]), np.min(X[y==1, 0]), np.max(X[y==1, 0])]

    x = np.arange(len(estadisticas))
    width = 0.35

    plt.bar(x - width/2, valores_normales, width, label='Normal', alpha=0.7, color='blue')
    plt.bar(x + width/2, valores_fallas, width, label='Falla', alpha=0.7, color='red')
    plt.xlabel('Estadísticas')
    plt.ylabel('Resistencia (Ω)')
    plt.title('Estadísticas de Resistencia')
    plt.xticks(x, estadisticas)
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Estadísticas descriptivas
    print("Estadísticas Descriptivas:")
    print("="*50)
    df_stats = pd.DataFrame(X, columns=nombres_caracteristicas)
    df_stats['Estado'] = ['Normal' if label == 0 else 'Falla' for label in y]
    print(df_stats.groupby('Estado').describe().round(3))
gráficos()

In [ ]:
# ============================================================================
# FUNCIÓN: Aplicar PCA para detección de fallas en circuitos
# ============================================================================
def aplicar_pca_detección_fallas(características, etiquetas, n_componentes):
    """
    Aplica PCA a datos de circuitos para reducir dimensionalidad y facilitar
    la detección de fallas mediante visualización y análisis de anomalías.

    Parámetros:
        características: Array (n x m) con m características de n circuitos
        etiquetas: Array (n) con etiquetas 0=normal, 1=falla
        n_componentes: Número de componentes principales a extraer

    Retorna:
        datos_pca: Array transformado en espacio PCA reducido
        modelo_pca: Modelo PCA ajustado (para transformaciones futuras)
        normalizador: Escalador ajustado (para normalizar nuevos datos)
        df_pca: DataFrame con componentes principales y etiquetas
    """

    # ========================================================================
    # PASO 1: Normalización de datos
    # ========================================================================
    # Estandarizar cada característica: media = 0, desviación estándar = 1
    # Esto es crucial porque las características tienen diferentes escalas
    # (ej: resistencia ~100Ω vs corriente de fuga ~1e-9A)
    normalizador = StandardScaler()
    características_normalizadas = normalizador.fit_transform(características)

    # Reemplazar cualquier NaN o Inf que pueda haber surgido
    características_normalizadas = np.nan_to_num(características_normalizadas, nan=0.0, posinf=0.0, neginf=0.0)

    # ========================================================================
    # PASO 2: Aplicar PCA
    # ========================================================================
    # Crear y ajustar modelo PCA con SVD para mayor estabilidad numérica
    modelo_pca = PCA(n_components=n_componentes, svd_solver='full')

    # Transformar datos al espacio de componentes principales
    # Cada componente es una combinación lineal de las características originales
    datos_pca = modelo_pca.fit_transform(características_normalizadas)

    # ========================================================================
    # PASO 3: Organizar resultados en DataFrame para análisis
    # ========================================================================
    # Crear DataFrame con componentes principales
    df_pca = pd.DataFrame(datos_pca, columns=[f'PC{i+1}' for i in range(n_componentes)])

    # Agregar columna con estado del circuito (Normal/Falla)
    df_pca['Estado'] = ['Normal' if etiq == 0 else 'Falla' for etiq in etiquetas]

    # Mostrar transformación de dimensiones
    print(f"Transformación: {características.shape} --> {características_normalizadas.shape} --> {datos_pca.shape}")
    print(f"Reducción: {características.shape[1]} características --> {datos_pca.shape[1]} componentes principales")

    return datos_pca, modelo_pca, normalizador, df_pca

# ============================================================================
# APLICAR PCA AL DATASET DE CIRCUITOS
# ============================================================================
# Reducir de 10 características a 3 componentes principales
X_pca, modelo_pca, normalizador, df_pca = aplicar_pca_detección_fallas(X, y, n_componentes=3)

In [ ]:
# para hacer pruebas
df_pca


In [ ]:
# ============================================================================
# VISUALIZACIÓN: Resultados del PCA en detección de fallas
# ============================================================================
def gráficos():
    """Visualiza los resultados del análisis PCA aplicado a datos de circuitos."""

    plt.figure(figsize=(15, 12))

    # ========================================================================
    # GRÁFICO 1: Varianza explicada por cada componente principal
    # ========================================================================
    plt.subplot(2, 3, 1)
    plt.bar(range(1, len(modelo_pca.explained_variance_ratio_)+1),
            modelo_pca.explained_variance_ratio_,
            color=['blue', 'green','brown'])
    plt.xlabel('Componente Principal')
    plt.ylabel('Varianza Explicada')
    plt.title('Varianza Explicada por Componentes')
    plt.xticks(range(1, len(modelo_pca.explained_variance_ratio_)+1),
               [f'PC{i+1}' for i in range(len(modelo_pca.explained_variance_ratio_))],
               rotation=45)
    plt.grid(True, alpha=0.3)

    # ========================================================================
    # GRÁFICO 2: Varianza acumulada
    # ========================================================================
    plt.subplot(2, 3, 2)
    varianza_acumulada = np.cumsum(modelo_pca.explained_variance_ratio_)
    plt.plot(range(1, len(varianza_acumulada)+1), varianza_acumulada, 'bo-')
    plt.xticks(range(1, len(varianza_acumulada)+1),
               [f'PC{i+1}' for i in range(len(varianza_acumulada))],
               rotation=45)
    plt.xlabel('Número de Componentes')
    plt.ylabel('Varianza Acumulada')
    plt.title('Varianza Acumulada')
    plt.grid(True, alpha=0.3)

    # ========================================================================
    # GRÁFICO 3: Scatter plot PC1 vs PC2
    # ========================================================================
    plt.subplot(2, 3, 3)
    # Crear máscaras para separar circuitos normales y defectuosos
    mascara_normal = df_pca['Estado'] == 'Normal'
    mascara_falla = df_pca['Estado'] == 'Falla'

    plt.scatter(df_pca[mascara_normal]['PC1'], df_pca[mascara_normal]['PC2'],
               alpha=0.6, label='Normal', color='blue')
    plt.scatter(df_pca[mascara_falla]['PC1'], df_pca[mascara_falla]['PC2'],
               alpha=0.6, label='Falla', color='red')
    plt.xlabel('Primera Componente Principal')
    plt.ylabel('Segunda Componente Principal')
    plt.title('PCA: PC1 vs PC2')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # ========================================================================
    # GRÁFICO 4: Scatter plot PC1 vs PC3
    # ========================================================================
    plt.subplot(2, 3, 4)
    plt.scatter(df_pca[mascara_normal]['PC1'], df_pca[mascara_normal]['PC3'],
               alpha=0.6, label='Normal', color='blue')
    plt.scatter(df_pca[mascara_falla]['PC1'], df_pca[mascara_falla]['PC3'],
               alpha=0.6, label='Falla', color='red')
    plt.xlabel('Primera Componente Principal')
    plt.ylabel('Tercera Componente Principal')
    plt.title('PCA: PC1 vs PC3')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # ========================================================================
    # GRÁFICO 5: Scatter plot PC2 vs PC3
    # ========================================================================
    plt.subplot(2, 3, 5)
    plt.scatter(df_pca[mascara_normal]['PC2'], df_pca[mascara_normal]['PC3'],
               alpha=0.6, label='Normal', color='blue')
    plt.scatter(df_pca[mascara_falla]['PC2'], df_pca[mascara_falla]['PC3'],
               alpha=0.6, label='Falla', color='red')
    plt.xlabel('Segunda Componente Principal')
    plt.ylabel('Tercera Componente Principal')
    plt.title('PCA: PC2 vs PC3')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # ========================================================================
    # GRÁFICO 6: Contribución de características originales a cada PC
    # ========================================================================
    plt.subplot(2, 3, 6)
    # Matriz de componentes: muestra cómo cada característica original
    # contribuye a cada componente principal
    componentes = modelo_pca.components_
    plt.imshow(componentes, cmap='coolwarm', aspect='auto')
    plt.colorbar()
    plt.xlabel('Características Originales')
    plt.ylabel('Componentes Principales')
    plt.title('Contribución de Características')
    plt.xticks(range(len(nombres_caracteristicas)), nombres_caracteristicas, rotation=90)

    plt.tight_layout()
    plt.show()

    # ========================================================================
    # RESUMEN NUMÉRICO
    # ========================================================================
    print(f"Varianza explicada por los primeros {len(modelo_pca.explained_variance_ratio_)} componentes:")
    for i, varianza in enumerate(modelo_pca.explained_variance_ratio_):
        print(f"  PC{i+1}: {varianza:.3f} ({varianza*100:.1f}%)")

    print(f"\nVarianza total explicada: {np.sum(modelo_pca.explained_variance_ratio_):.3f} ({np.sum(modelo_pca.explained_variance_ratio_)*100:.1f}%)")

gráficos()

In [ ]:
# ============================================================================
# FUNCIÓN: Detección de anomalías usando Isolation Forest
# ============================================================================
def detectar_anomalías_pca(datos_pca, etiquetas_reales, contaminación=0.1):
    """
    Detecta anomalías (circuitos defectuosos) usando el algoritmo Isolation Forest
    aplicado sobre los datos en el espacio PCA reducido.

    Isolation Forest es un algoritmo de detección de anomalías que identifica
    puntos de datos aislados (outliers) que se separan fácilmente del resto.

    Parámetros:
        datos_pca: Array con datos en espacio PCA reducido
        etiquetas_reales: Array con etiquetas verdaderas (0=normal, 1=falla)
        contaminación: Proporción esperada de anomalías en los datos (default: 0.1)

    Retorna:
        predicciones: Array con predicciones (0=normal, 1=anomalía)
        precisión: Precisión del modelo (fracción de predicciones correctas)
        modelo_bosque: Modelo Isolation Forest entrenado
    """

    # ========================================================================
    # ENTRENAR MODELO ISOLATION FOREST
    # ========================================================================
    # Crear modelo con semilla aleatoria fija para reproducibilidad
    modelo_bosque = IsolationForest(contamination=contaminación, random_state=42)

    # Entrenar y predecir en un solo paso
    # Retorna: -1 para anomalías, +1 para datos normales
    anomalias = modelo_bosque.fit_predict(datos_pca)

    # ========================================================================
    # CONVERTIR PREDICCIONES AL FORMATO ESPERADO
    # ========================================================================
    # Convertir de formato Isolation Forest (-1/+1) a formato binario (1/0)
    # -1 (anomalía) --> 1
    # +1 (normal)   --> 0
    predicciones = (anomalias == -1).astype(int)

        # ========================================================================
    # CALCULAR MÉTRICAS DE DESEMPEÑO
    # ========================================================================
    from sklearn.metrics import accuracy_score

    # Precisión: porcentaje de predicciones correctas
    precisión = accuracy_score(etiquetas_reales, predicciones)

    return predicciones, precisión, modelo_bosque

# ============================================================================
# APLICAR DETECCIÓN DE ANOMALÍAS
# ============================================================================
predicciones, precisión, modelo_bosque = detectar_anomalías_pca(X_pca, y, contaminación=0.1)

print(f"Precisión del modelo: {precisión:.3f} ({precisión*100:.1f}%)")
print(f"El modelo detectó correctamente {precisión*100:.1f}% de los circuitos")

---

<a id="compresion"></a>
# 📸 Aplicación 3: Compresión de Imágenes de Componentes Electrónicos [↑](#inicio)

---

## 🎯 Problema

En sistemas de inspección automática de componentes electrónicos, las imágenes de alta resolución ocupan mucho espacio de almacenamiento:
- 💾 Grandes volúmenes de datos
- ⚡ Procesamiento lento
- 📡 Transmisión costosa
- 🔍 Necesidad de mantener calidad para análisis

## 💡 Solución con PCA

Aplicamos PCA para compresión de imágenes manteniendo las características más importantes:
1. Vectorización de imágenes (píxeles → features)
2. Reducción dimensional controlada
3. Reconstrucción con pérdida mínima


In [ ]:
# ============================================================================
# FUNCIONES: Generación de imágenes sintéticas de componentes electrónicos
# ============================================================================

def resistencia(tamaño=(64, 64)):
    """
    Genera una imagen sintética de una resistencia electrónica.

    Dibuja un rectángulo horizontal (cuerpo de la resistencia) con cables
    en ambos extremos. Con 20% de probabilidad, añade defectos visuales.

    Parámetros:
        tamaño: Tupla (alto, ancho) con dimensiones de la imagen en píxeles

    Retorna:
        imagen: Array 2D (matriz) con valores 0-1 representando la imagen
        es_defectuoso: Booleano indicando si tiene defectos
    """
    # Crear imagen negra (todos los píxeles en 0)
    imagen = np.zeros(tamaño)

    # ========================================================================
    # DIBUJAR CUERPO DE LA RESISTENCIA
    # ========================================================================
    centro_x, centro_y = tamaño[0]//2, tamaño[1]//2
    longitud = 40  # Longitud del cuerpo en píxeles
    ancho = 8      # Ancho del cuerpo en píxeles

    # Calcular coordenadas del rectángulo del cuerpo
    y_inicio = centro_y - ancho//2
    y_fin = centro_y + ancho//2
    x_inicio = centro_x - longitud//2
    x_fin = centro_x + longitud//2

    # Dibujar cuerpo: píxeles blancos (valor 1)
    imagen[y_inicio:y_fin, x_inicio:x_fin] = 1

    # ========================================================================
    # DIBUJAR CABLES TERMINALES
    # ========================================================================
    # Cable izquierdo: desde el borde hasta el inicio del cuerpo
    imagen[centro_y-2:centro_y+2, :x_inicio] = 1
    # Cable derecho: desde el final del cuerpo hasta el borde
    imagen[centro_y-2:centro_y+2, x_fin:] = 1

    # ========================================================================
    # AÑADIR DEFECTOS VISUALES ALEATORIOS (20% de probabilidad)
    # ========================================================================
    es_defectuoso = np.random.random() < 0.2

    if es_defectuoso:
        if np.random.random() < 0.5:
            # DEFECTO TIPO 1: Grieta o raya en el cuerpo
            # Crear línea oscura (valor 0) simulando daño
            imagen[centro_y-1:centro_y+1, x_inicio+10:x_inicio+15] = 0
        else:
            # DEFECTO TIPO 2: Mancha o quemadura
            # Posición aleatoria cerca del centro
            mancha_x = centro_x + np.random.randint(-10, 10)
            mancha_y = centro_y + np.random.randint(-5, 5)

            # Verificar que la mancha esté dentro de los límites de la imagen
            if 0 <= mancha_x < tamaño[0] and 0 <= mancha_y < tamaño[1]:
                # Dibujar mancha gris (valor 0.5)
                imagen[mancha_y-2:mancha_y+2, mancha_x-2:mancha_x+2] = 0.5

    return imagen, es_defectuoso
def capacitor(tamaño=(64, 64)):
    """
    Genera una imagen sintética de un capacitor electrolítico.

    Dibuja un rectángulo vertical (cuerpo cilíndrico del capacitor) con cables
    en ambos lados. Con 20% de probabilidad, añade defectos visuales.

    Parámetros:
        tamaño: Tupla (alto, ancho) con dimensiones de la imagen en píxeles

    Retorna:
        imagen: Array 2D con valores 0-1 representando la imagen
        es_defectuoso: Booleano indicando si tiene defectos
    """
    # Crear imagen negra (fondo)
    imagen = np.zeros(tamaño)

    # ========================================================================
    # DIBUJAR CUERPO DEL CAPACITOR
    # ========================================================================
    centro_x, centro_y = tamaño[0]//2, tamaño[1]//2
    ancho = 20  # Ancho del cuerpo (más ancho que la resistencia)
    alto = 30   # Alto del cuerpo (vertical)

    # Calcular coordenadas del rectángulo del cuerpo
    y_inicio = centro_y - alto//2
    y_fin = centro_y + alto//2
    x_inicio = centro_x - ancho//2
    x_fin = centro_x + ancho//2

    # Dibujar cuerpo cilíndrico: píxeles blancos
    imagen[y_inicio:y_fin, x_inicio:x_fin] = 1

    # ========================================================================
    # DIBUJAR CABLES TERMINALES
    # ========================================================================
    # Cable izquierdo (terminal negativo)
    imagen[centro_y-2:centro_y+2, :x_inicio] = 1
    # Cable derecho (terminal positivo)
    imagen[centro_y-2:centro_y+2, x_fin:] = 1

    # ========================================================================
    # AÑADIR DEFECTOS VISUALES ALEATORIOS (20% de probabilidad)
    # ========================================================================
    es_defectuoso = np.random.random() < 0.2

    if es_defectuoso:
        if np.random.random() < 0.5:
            # DEFECTO TIPO 1: Deformación o abultamiento del cuerpo
            # Típico en capacitores a punto de explotar
            # Cambiar tono del interior (valor 0.8 = gris claro)
            imagen[y_inicio+5:y_fin-5, x_inicio+2:x_fin-2] = 0.8
        else:
            # DEFECTO TIPO 2: Grieta horizontal
            # Simulando daño físico o sobrecalentamiento
            imagen[y_inicio+10:y_inicio+12, x_inicio:x_fin] = 0

    return imagen, es_defectuoso
def inductor(tamaño=(64, 64)):
    """
    Genera una imagen sintética de un inductor (bobina).

    Dibuja un anillo circular que representa la bobina del inductor, con cables
    conectados a ambos lados. Con 20% de probabilidad, añade defectos visuales.

    Parámetros:
        tamaño: Tupla (alto, ancho) con dimensiones de la imagen en píxeles

    Retorna:
        imagen: Array 2D con valores 0-1 representando la imagen
        es_defectuoso: Booleano indicando si tiene defectos
    """
    # Crear imagen negra (fondo)
    imagen = np.zeros(tamaño)

    # ========================================================================
    # DIBUJAR BOBINA CIRCULAR
    # ========================================================================
    centro_x, centro_y = tamaño[0]//2, tamaño[1]//2
    radio = 15  # Radio de la bobina en píxeles

    # Crear máscara circular usando coordenadas cartesianas
    y, x = np.ogrid[:tamaño[0], :tamaño[1]]
    # Calcular distancia de cada píxel al centro
    distancia = np.sqrt((x - centro_x)**2 + (y - centro_y)**2)

    # Crear anillo: píxeles entre radio-3 y radio
    # Esto crea un círculo hueco que representa el alambre enrollado
    mascara = (distancia <= radio) & (distancia >= radio-3)
    imagen[mascara] = 1

    # ========================================================================
    # DIBUJAR CABLES TERMINALES
    # ========================================================================
    # Cable izquierdo: desde el borde hasta el lado izquierdo de la bobina
    imagen[centro_y-2:centro_y+2, :centro_x-radio+1] = 1
    # Cable derecho: desde el lado derecho de la bobina hasta el borde
    imagen[centro_y-2:centro_y+2, centro_x+radio:] = 1

    # ========================================================================
    # AÑADIR DEFECTOS VISUALES ALEATORIOS (20% de probabilidad)
    # ========================================================================
    es_defectuoso = np.random.random() < 0.2

    if es_defectuoso:
        # DEFECTO: Interrupción en la bobina
        # Simula un corte en el alambre del inductor

        # Seleccionar posición angular aleatoria para el defecto
        angulo = np.random.uniform(0, 2*np.pi)

        # Convertir coordenadas polares a cartesianas
        x_defecto = int(centro_x + (radio-1.5) * np.cos(angulo))
        y_defecto = int(centro_y + (radio-1.5) * np.sin(angulo))

        # Verificar que el defecto esté dentro de los límites
        if 0 <= x_defecto < tamaño[0] and 0 <= y_defecto < tamaño[1]:
            # Crear pequeño hueco (píxeles negros) en la bobina
            imagen[y_defecto-1:y_defecto+1, x_defecto-1:x_defecto+1] = 0

    return imagen, es_defectuoso
def chip(tamaño=(64, 64)):
    """
    Genera una imagen sintética de un chip de circuito integrado (IC).

    Dibuja un rectángulo horizontal (cuerpo del chip) con 8 pines (terminales)
    en la parte superior e inferior. Con 20% de probabilidad, añade defectos.

    Parámetros:
        tamaño: Tupla (alto, ancho) con dimensiones de la imagen en píxeles

    Retorna:
        imagen: Array 2D con valores 0-1 representando la imagen
        es_defectuoso: Booleano indicando si tiene defectos
    """
    # Crear imagen negra (fondo)
    imagen = np.zeros(tamaño)

    # ========================================================================
    # DIBUJAR CUERPO DEL CHIP
    # ========================================================================
    centro_x, centro_y = tamaño[0]//2, tamaño[1]//2
    ancho = 34  # Ancho del cuerpo del chip
    alto = 20   # Alto del cuerpo del chip

    # Calcular coordenadas del rectángulo del cuerpo
    y_inicio = centro_y - alto//2
    y_fin = centro_y + alto//2
    x_inicio = centro_x - ancho//2 + 1
    x_fin = centro_x + ancho//2

    # Dibujar cuerpo del chip: píxeles blancos
    imagen[y_inicio:y_fin, x_inicio:x_fin] = 1

    # ========================================================================
    # DIBUJAR PINES (TERMINALES)
    # ========================================================================
    # Dibujar 8 pines: 4 arriba y 4 abajo
    # Estilo DIP (Dual In-line Package)
    for indice_pin in range(8):
        # Calcular posición X de cada pin (distribuidos uniformemente)
        pin_x = x_inicio + 2 + (indice_pin * ancho) // 8

        # Pin superior (hacia arriba desde el cuerpo)
        imagen[y_inicio-3:y_inicio, pin_x-1:pin_x+1] = 1
        # Pin inferior (hacia abajo desde el cuerpo)
        imagen[y_fin:y_fin+3, pin_x-1:pin_x+1] = 1

    # ========================================================================
    # AÑADIR DEFECTOS VISUALES ALEATORIOS (20% de probabilidad)
    # ========================================================================
    es_defectuoso = np.random.random() < 0.2

    if es_defectuoso:
        # DEFECTO: Pin faltante o roto
        # Simula soldadura deficiente o daño mecánico

        # Seleccionar aleatoriamente cuál pin está dañado (0-7)
        pin_faltante = np.random.randint(0, 8)
        pin_x = x_inicio + 2 + (pin_faltante * ancho // 8)

        # Verificar límites de la imagen
        if pin_x < tamaño[0]:
            # Borrar el pin superior e inferior (ponerlos en negro)
            imagen[y_inicio-3:y_inicio, pin_x-1:pin_x+1] = 0
            imagen[y_fin:y_fin+3, pin_x-1:pin_x+1] = 0

    return imagen, es_defectuoso

In [ ]:
# ============================================================================
# VISUALIZACIÓN DE PRUEBA: Ejemplos de cada tipo de componente
# ============================================================================
def mostrar_imágenes_de_componentes():
    """Muestra un ejemplo de cada tipo de componente electrónico."""

    plt.figure(figsize=(15, 10))

    # Definir tipos de componentes
    tipos_componentes = ["Resistencia", "Capacitor", "Inductor", "Chip"]

    # Generar un ejemplo de cada tipo
    imagenes_componentes = [resistencia(), capacitor(), inductor(), chip()]

    # Mostrar cada componente
    for i in range(4):
        imagen, es_defectuoso = imagenes_componentes[i]
        plt.subplot(2, 4, i+1)
        plt.imshow(imagen, cmap='gray')
        plt.title(f'{tipos_componentes[i]}')
        estado = "defectuoso" if es_defectuoso else "bueno"
        plt.xlabel(f'{estado}')
        plt.xticks([]), plt.yticks([])

    plt.tight_layout()
    plt.show()

mostrar_imágenes_de_componentes()

In [ ]:
# ============================================================================
# FUNCIÓN: Generación de dataset de imágenes de componentes
# ============================================================================
def generar_imágenes_componentes_electrónicos(n_imágenes=100, tamaño=(64, 64)):
    """
    Genera un dataset de imágenes sintéticas de componentes electrónicos.

    Crea aleatoriamente imágenes de 4 tipos de componentes (resistencia,
    capacitor, inductor, chip), añade ruido realista, y las convierte
    a formato vectorial para análisis con PCA.

    Parámetros:
        n_imágenes: Cantidad de imágenes a generar (default: 100)
        tamaño: Tupla (alto, ancho) de cada imagen en píxeles (default: 64x64)

    Retorna:
        imágenes: Array (n x píxeles) donde cada fila es una imagen aplanada
        etiquetas: Array (n) con etiquetas 1=defectuoso, 0=bueno
    """
        # Fijar semilla para reproducibilidad
    np.random.seed(42)

    # Listas para almacenar imágenes y etiquetas
    imágenes = []       # Lista de vectores (imágenes aplanadas)
    etiquetas = []      # Lista de etiquetas binarias

    # ========================================================================
    # GENERAR CADA IMAGEN
    # ========================================================================
    for i in range(n_imágenes):
        # Seleccionar aleatoriamente tipo de componente
        tipo_componente = np.random.choice(['resistencia', 'capacitor', 'inductor', 'chip'])

        # Generar imagen del componente correspondiente
        if tipo_componente == 'resistencia':
            imagen, es_defectuoso = resistencia()
        elif tipo_componente == 'capacitor':
            imagen, es_defectuoso = capacitor()
        elif tipo_componente == 'inductor':
            imagen, es_defectuoso = inductor()
        elif tipo_componente == 'chip':
            imagen, es_defectuoso = chip()

        # ====================================================================
        # AÑADIR RUIDO GAUSSIANO REALISTA
        # ====================================================================
        # Simula imperfecciones del sensor de la cámara y condiciones de iluminación
        ruido = np.random.normal(0, 0.1, tamaño)

        # Sumar ruido y recortar valores para mantenerlos en rango [0, 1]
        imagen = np.clip(imagen + ruido, 0, 1)

                # ====================================================================
        # APLANAR IMAGEN Y GUARDAR
        # ====================================================================
        # Convertir matriz 2D (64x64) a vector 1D (4096 elementos)
        # Esto es necesario para procesamiento con PCA
        imágenes.append(imagen.flatten())

        # Guardar etiqueta: 1=defectuoso, 0=bueno
        etiquetas.append(1 if es_defectuoso else 0)

    # ========================================================================
    # RESUMEN DEL DATASET GENERADO
    # ========================================================================
    print(f"Imágenes generadas: {len(imágenes)}")
    print(f"Dimensiones de cada imagen: {tamaño[0]}x{tamaño[1]}")
    print(f"Formato: {len(imágenes)} x {tamaño} --> {len(imágenes)} x {tamaño[0]*tamaño[1]}")
    print(f"Componentes defectuosos: {np.sum(etiquetas)} ({np.sum(etiquetas)/len(etiquetas)*100:.1f}%)")
    print(f"Componentes buenos: {len(etiquetas) - np.sum(etiquetas)} ({(len(etiquetas) - np.sum(etiquetas))/len(etiquetas)*100:.1f}%)")

    # Convertir listas a arrays de NumPy
    return np.array(imágenes), np.array(etiquetas)

# ============================================================================
# GENERAR DATASET DE 100 IMÁGENES
# ============================================================================
imágenes_componentes, etiquetas_componentes = generar_imágenes_componentes_electrónicos()

In [ ]:
# Para hacer pruebas


In [ ]:
# ============================================================================
# VISUALIZACIÓN: Muestra de imágenes generadas
# ============================================================================
def gráficos():
    """Visualiza una muestra aleatoria de imágenes de componentes."""

    plt.figure(figsize=(15, 10))

        # ========================================================================
    # MOSTRAR 12 IMÁGENES ALEATORIAS DEL DATASET
    # ========================================================================
    # Seleccionar 12 índices aleatorios sin reemplazo
    índices_aleatorios = np.random.choice(range(len(imágenes_componentes)), 12, replace=False)

    # Mostrar cada imagen en un subplot
    for i, índice in enumerate(índices_aleatorios):
        plt.subplot(3, 4, i+1)

        # Convertir vector 1D (4096) de vuelta a matriz 2D (64x64)
        imagen_ejemplo = imágenes_componentes[índice].reshape(64, 64)

        # Mostrar imagen en escala de grises
        plt.imshow(imagen_ejemplo, cmap='gray')

        # Título con número de imagen y estado
        estado = "Defectuoso" if etiquetas_componentes[índice] else "Bueno"
        plt.title(f'Imagen {índice+1} - {estado}')
        plt.axis('off')  # Ocultar ejes

    plt.tight_layout()
    plt.show()

gráficos()

In [ ]:
# ============================================================================
# FUNCIÓN: Aplicar PCA a imágenes para compresión y extracción de características
# ============================================================================
def aplicar_pca_imágenes(imágenes, n_componentes=50):
    """
    Aplica PCA a un conjunto de imágenes para reducir dimensionalidad.

    Transforma imágenes de alta dimensión (ej: 4096 píxeles) a un espacio
    de características reducido conservando la mayor varianza posible.
    Útil para compresión, visualización y clasificación.

    Parámetros:
        imágenes: Array (n x píxeles) con n imágenes aplanadas
        n_componentes: Número de componentes principales a conservar (default: 50)

    Retorna:
        imágenes_pca: Array (n x n_componentes) en espacio reducido
        modelo_pca: Modelo PCA ajustado (para reconstrucción)
        normalizador: Escalador ajustado (para normalizar nuevas imágenes)
    """

    # ========================================================================
    # PASO 1: Normalización de píxeles
    # ========================================================================
    # Estandarizar cada píxel (columna): media = 0, desviación estándar = 1
    # Esto asegura que todos los píxeles contribuyan equitativamente al PCA
    normalizador = StandardScaler()
    imágenes_normalizadas = normalizador.fit_transform(imágenes)

    # Reemplazar cualquier NaN o Inf que pueda haber surgido
    imágenes_normalizadas = np.nan_to_num(imágenes_normalizadas, nan=0.0, posinf=0.0, neginf=0.0)

    # ========================================================================
    # PASO 2: Aplicar PCA para reducir dimensionalidad
    # ========================================================================
    # Crear modelo PCA que conservará n_componentes principales
    # Usar SVD completo para mayor estabilidad numérica
    modelo_pca = PCA(n_components=n_componentes, svd_solver='full')

    # Ajustar modelo y transformar imágenes al espacio reducido
    # Esto encuentra las direcciones de máxima varianza en el dataset
    imágenes_pca = modelo_pca.fit_transform(imágenes_normalizadas)

    # ========================================================================
    # RESUMEN DE LA TRANSFORMACIÓN
    # ========================================================================
    print(f"Transformación: {len(imágenes)} x {imágenes_componentes.shape[1]} → {len(imágenes)} x {imágenes_pca.shape[1]}")
    print(f"Reducción de dimensionalidad: {imágenes_componentes.shape[1]} → {imágenes_pca.shape[1]}")
    print(f"Varianza explicada total: {np.sum(modelo_pca.explained_variance_ratio_):.3f} ({np.sum(modelo_pca.explained_variance_ratio_)*100:.1f}%)")
    print(f"Factor de compresión: {imágenes_componentes.shape[1]/imágenes_pca.shape[1]:.1f}x")
    print(f"Ahorro de espacio: {(1 - imágenes_pca.shape[1]/imágenes_componentes.shape[1])*100:.1f}%")

    return imágenes_pca, modelo_pca, normalizador

# ============================================================================
# APLICAR PCA AL DATASET DE IMÁGENES
# ============================================================================
# Reducir de 4096 píxeles a 50 componentes principales (compresión ~82x)
imágenes_pca, modelo_pca_imágenes, normalizador_imágenes = aplicar_pca_imágenes(imágenes_componentes, n_componentes=50)

In [ ]:
# ============================================================================
# VISUALIZACIÓN: Resultados del PCA en imágenes
# ============================================================================
def gráficos():
    """Visualiza resultados de compresión PCA aplicado a imágenes."""

    plt.figure(figsize=(15, 12))

    # ========================================================================
    # GRÁFICO 1: Varianza explicada por cada componente principal
    # ========================================================================
    plt.subplot(2, 3, 1)
    plt.bar(range(1, len(modelo_pca_imágenes.explained_variance_ratio_)+1),
            modelo_pca_imágenes.explained_variance_ratio_)
    plt.xlabel('Componente Principal')
    plt.ylabel('Varianza Explicada')
    plt.title('Varianza Explicada por Componentes')
    plt.grid(True, alpha=0.3)

    # ========================================================================
    # GRÁFICO 2: Imagen original (primera del dataset)
    # ========================================================================
    plt.subplot(2, 3, 2)
    imagen_original = imágenes_componentes[0].reshape(64, 64)
    plt.imshow(imagen_original, cmap='gray')
    plt.title('Imagen Original')
    plt.axis('off')

    # ========================================================================
    # GRÁFICO 3: Imagen reconstruida desde PCA
    # ========================================================================
    # Reconstruir imagen desde componentes principales
    # Proceso: PCA inverso -> desnormalizar -> reshape
    imagen_reconstruida = modelo_pca_imágenes.inverse_transform(imágenes_pca[0:1])
    imagen_reconstruida = normalizador_imágenes.inverse_transform(imagen_reconstruida)
    imagen_reconstruida = imagen_reconstruida.reshape(64, 64)

    plt.subplot(2, 3, 3)
    plt.imshow(imagen_reconstruida, cmap='gray')
    plt.title('Imagen Reconstruida (PCA)')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    # ========================================================================
    # MÉTRICAS DE CALIDAD DE RECONSTRUCCIÓN
    # ========================================================================
    # Calcular error cuadrático medio (MSE) entre originales y reconstruidas
    imágenes_reconstruidas_completas = normalizador_imágenes.inverse_transform(
        modelo_pca_imágenes.inverse_transform(imágenes_pca)
    )
    error_reconstrucción = np.mean((imágenes_componentes - imágenes_reconstruidas_completas)**2)

    print(f"Métricas de PCA en imágenes:")
    print(f"=" * 50)
    print(f"Error de reconstrucción (MSE): {error_reconstrucción:.6f}")
    print(f"Varianza total explicada: {np.sum(modelo_pca_imágenes.explained_variance_ratio_):.3f} ({np.sum(modelo_pca_imágenes.explained_variance_ratio_)*100:.1f}%)")
    print(f"Factor de compresión: {imágenes_componentes.shape[1]/imágenes_pca.shape[1]:.1f}x")
    print(f"Reducción de dimensionalidad: {imágenes_componentes.shape[1]} → {imágenes_pca.shape[1]}")
    print(f"Ahorro de memoria: {(1 - imágenes_pca.shape[1]/imágenes_componentes.shape[1])*100:.1f}%")

gráficos()

In [ ]:
# Para hacer pruebas


---

<a id="recomendaciones"></a>
## 📌 Buenas Prácticas al Usar PCA en Electrónica  [↑](#inicio)

---

### Ventajas del PCA en Electrónica

- **Eficiencia computacional**: Reduce la complejidad de los algoritmos
- **Robustez**: Mejora la tolerancia al ruido
- **Interpretabilidad**: Facilita la comprensión de los datos
- **Escalabilidad**: Funciona con grandes volúmenes de datos
- **Versatilidad**: Se aplica a diferentes tipos de datos electrónicos

### Limitaciones y Consideraciones

- **Pérdida de información**: Al reducir dimensiones se puede perder información
- **Linealidad**: PCA asume relaciones lineales entre variables
- **Sensibilidad a outliers**: Los valores extremos pueden afectar los resultados
- **Interpretación**: Los componentes principales pueden ser difíciles de interpretar

### Recomendaciones Prácticas

- **Preprocesamiento**: Normalizar siempre los datos antes de aplicar PCA
- **Selección de componentes**: Usar el criterio de varianza explicada (el 95% es común)
- **Validación**: Verificar que la reducción de dimensionalidad no afecte la calidad de los resultados
- **Visualización**: Usar las primeras componentes para visualización 2D/3D
- **Combinación**: El PCA funciona bien combinado con otros algoritmos de aprendizaje automático

### Casos de Uso Reales

- **Sistemas de Monitoreo Industrial**
  - Monitoreo de vibraciones en motores eléctricos
  - Análisis de temperatura en transformadores
  - Detección de fallas en sistemas de potencia
  
- **Dispositivos IoT y Sensores**
  - Compresión de datos en sensores remotos
  - Análisis de patrones en redes de sensores
  - Optimización de transmisión de datos
  
- **Control de Calidad en Manufactura**
  - Inspección automática de componentes
  - Detección de defectos en PCB
  - Análisis de calidad en líneas de producción
  
- **Sistemas de Diagnóstico**
  - Diagnóstico de fallas en equipos médicos
  - Análisis de señales biomédicas
  - Monitoreo de sistemas críticos

### Métricas de Evaluación

- **Varianza explicada**: Porcentaje de información conservada
- **Error de reconstrucción**: Diferencia entre los datos originales y los reconstruidos
- **Ratio de compresión**: Reducción en el número de dimensiones
- **Tiempo de procesamiento**: Eficiencia computacional

### Extensiones y Variantes

- **Kernel PCA**: Para datos no lineales
- **Incremental PCA**: Para datos en streaming
- **Sparse PCA**: Para datos con muchas características irrelevantes
- **Robust PCA**: Para datos con outliers

---

<a id="referencias"></a>
# 📚 Recursos Adicionales [↑](#inicio)

---

- Scikit-learn Documentation \
  https://scikit-learn.org/stable/modules/decomposition.html#pca
- Pattern Recognition and Machine Learning - Christopher Bishop
- The Elements of Statistical Learning - Hastie, Tibshirani, Friedman
- Papers sobre PCA en aplicaciones electrónicas y de ingenierí

